In [1]:
import dotenv
dotenv.load_dotenv()

In [2]:
import yaml
from datasets import Dataset
from trl import CPOTrainer, CPOConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig

[2024-06-13 03:53:43,665] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [3]:
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'
checkpoint = yaml.safe_load(open('checkpoint.yaml'))
dataset = Dataset.from_dict(
    {
        'prompt': checkpoint['prompts'],
        'chosen': checkpoint['chosen'],
        'rejected': checkpoint['rejected'],
    }
)

In [4]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype='bfloat16',
) 

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

split = dataset.train_test_split(test_size=0.1)
train_dataset = split['train']
test_dataset = split['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
trainer = CPOTrainer(
    model,
    args=CPOConfig(
        output_dir='output',
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        bf16=True,
        gradient_accumulation_steps=4, 
        do_eval=True,
        logging_strategy='steps',
        save_strategy='steps',
        evaluation_strategy='steps',
        logging_steps=100,
        save_steps=100,
        eval_steps=100,
        num_train_epochs=5,
        load_best_model_at_end=True,
        warmup_steps=200,
    ),
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/cpo_trainer.py:202: UserWarning: `max_length` is not set in the CPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/cpo_trainer.py:211: UserWarning: `max_prompt_length` is not set in the CPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/cpo_trainer.py:240: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/527 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [6]:
trainer.train()

wandb: Currently logged in as: chengpong1127. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss
100,3.228500,2.363034,24.833600,2.376000,2.376000,-11.158219,-11.482299,0.525424,0.324082,-114.823013,-111.582184,-0.170178,-0.233762,0.493841
200,2.453900,1.932886,25.089200,2.352000,2.352000,-10.361959,-11.242377,0.593220,0.880419,-112.423767,-103.619576,-0.196516,-0.245825,0.461277
300,1.474900,1.864904,25.165800,2.344000,2.344000,-9.611459,-11.275692,0.711864,1.664232,-112.756912,-96.114586,-0.314029,-0.329035,0.416869
400,0.891200,1.754683,25.127600,2.348000,2.348000,-9.470873,-11.713627,0.711864,2.242754,-117.136261,-94.708725,-0.393390,-0.398838,0.409609
500,0.539000,1.782290,24.773300,2.382000,2.382000,-9.531375,-12.055165,0.711864,2.523789,-120.551636,-95.313751,-0.416427,-0.412111,0.407875
600,0.375400,1.803838,24.124400,2.446000,2.446000,-9.148738,-11.529034,0.711864,2.380296,-115.290344,-91.487389,-0.377285,-0.369473,0.390640


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-666a6ec1-364a1c905f280a1101d2434c;d7aa341c-d873-4765-8bb9-c545239cf42b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-666a703a-5e354933744bb0d

TrainOutput(global_step=655, training_loss=1.4022609856292492, metrics={'train_runtime': 2416.8219, 'train_samples_per_second': 1.09, 'train_steps_per_second': 0.271, 'total_flos': 0.0, 'train_loss': 1.4022609856292492, 'epoch': 4.97})